In [1]:
import pandas as pd
import random

from faker import Faker

import sqlite3

%load_ext sql

In [2]:
N_PRODUTOS = 100
N_CLIENTES = 20
N_COMPRAS = 500

In [3]:
faker = Faker('pt_BR')
faker.name()

'Sarah Campos'

In [4]:
db = sqlite3.connect('TesteSQL.db')
db

In [5]:
%sql sqlite:///TesteSQL.db

In [6]:
produtos = pd.DataFrame([[f'prod {i:02}', round(random.random() * 10, 2)] 
                         for i in range(N_PRODUTOS)], 
                        columns=['produto', 'preço'])
produtos.head()

,produto,preço
0,prod 00,8.35
1,prod 01,9.53
2,prod 02,3.27
3,prod 03,7.43
4,prod 04,0.22


In [7]:
produtos.to_sql('produtos', db, index_label='id', if_exists='replace')

In [8]:
pd.read_sql("SELECT * FROM produtos;", db, 'id').head()

,produto,preço
id,,
0,prod 00,8.35
1,prod 01,9.53
2,prod 02,3.27
3,prod 03,7.43
4,prod 04,0.22


In [9]:
clientes = pd.DataFrame([
    [faker.name(), faker.free_email(), random.randint(18, 60)]
for _ in range(N_CLIENTES)], columns=['nome', 'email', 'idade'])

clientes.head()

,nome,email,idade
0,Isaac Monteiro,caldeiraandre@yahoo.com.br,38
1,Letícia das Neves,cardosohelena@gmail.com,21
2,Dr. João Gabriel Araújo,carvalholuana@hotmail.com,52
3,Pedro Henrique Ferreira,pedro93@uol.com.br,58
4,Alexia Pinto,lpereira@gmail.com,21


In [10]:
clientes.to_sql('clientes', db, index_label='id', if_exists='replace')
pd.read_sql("SELECT * FROM clientes;", db, 'id').head()

,nome,email,idade
id,,,
0,Isaac Monteiro,caldeiraandre@yahoo.com.br,38
1,Letícia das Neves,cardosohelena@gmail.com,21
2,Dr. João Gabriel Araújo,carvalholuana@hotmail.com,52
3,Pedro Henrique Ferreira,pedro93@uol.com.br,58
4,Alexia Pinto,lpereira@gmail.com,21


In [11]:
compras = pd.DataFrame([[
    random.randint(0, N_CLIENTES - 1),
    random.randint(0, N_PRODUTOS - 1)
] for _ in range(N_COMPRAS)], columns=['id_cliente', 'id_produto'])
compras.head()

,id_cliente,id_produto
0,17,7
1,11,74
2,0,65
3,12,20
4,13,50


In [12]:
compras.to_sql('compras', db, index_label='id', if_exists='replace')
pd.read_sql("SELECT * FROM compras;", db, 'id').head()

,id_cliente,id_produto
id,,
0,17,7
1,11,74
2,0,65
3,12,20
4,13,50


In [13]:
del compras, clientes, produtos

# Select

Ao fazer uma extração num banco de dados SQL, independentemente do 
DBMS (se é Oracle, SQLite, MySQL/MariaDB e etc.), você sempre vai utilizar a 
palava `SELECT` seguida pelo que você quer trazer, e por fim a palavra FROM
informando de onde você irá coletar esses dados, por exemplo:

In [14]:
%%sql

SELECT nome, idade FROM clientes

 * sqlite:///TesteSQL.db
Done.


nome,idade
Isaac Monteiro,38
Letícia das Neves,21
Dr. João Gabriel Araújo,52
Pedro Henrique Ferreira,58
Alexia Pinto,21
Maria Fernanda Silveira,37
Lucca Vieira,25
Vitor Hugo Moraes,38
Srta. Sophie Pires,19
Luigi Moreira,22


Nesse caso, eu estou trazendo o nome e idade de todos os clientes da base.

# Limit

Geralmente não queremos trazer todos os dados de uma vez, muitas vezes os dados
que guardamos nas tabelas do banco de dados iriam facilmente estourar a  memória
dos nossos servidores, para limitar a pesquisa, existem vários métodos, o mais 
simples que tem é o `LIMIT n`, onde você informa quantos registros você quer ver
no máximo.

In [15]:
%%sql

SELECT * FROM clientes LIMIT 5

 * sqlite:///TesteSQL.db
Done.


id,nome,email,idade
0,Isaac Monteiro,caldeiraandre@yahoo.com.br,38
1,Letícia das Neves,cardosohelena@gmail.com,21
2,Dr. João Gabriel Araújo,carvalholuana@hotmail.com,52
3,Pedro Henrique Ferreira,pedro93@uol.com.br,58
4,Alexia Pinto,lpereira@gmail.com,21


Nesse caso, eu trouxe 5 registros da tabela cliente.

<span style="color:red"><b>Em uma aplicação, sempre use o</span></b> `LIMIT` 
<span style="color:red"><b>com um</span></b> `ORDER BY` 
<span style="color:red"><b>quer será explicado mais a frente

## Exercício 01

Traga 10 produtos e seu respectivo preço

In [16]:
%%sql

SELECT * FROM produtos LIMIT 10

 * sqlite:///TesteSQL.db
Done.


id,produto,preço
0,prod 00,8.35
1,prod 01,9.53
2,prod 02,3.27
3,prod 03,7.43
4,prod 04,0.22
5,prod 05,6.5
6,prod 06,9.81
7,prod 07,1.94
8,prod 08,6.78
9,prod 09,6.52


# Projeção

Também posso operar sobre os valores retornados, por exemplo, se ao invés da idade
da pessoa, eu quiser saber somente a faixa etária dela, posso fazer algum cálculo
no select e assim trazer esse dado já trabalhado do próprio banco de dados. 

In [17]:
%%sql

SELECT NOME, ROUND(idade / 10) * 10 AS FAIXA_ETARIA
FROM clientes

 * sqlite:///TesteSQL.db
Done.


nome,FAIXA_ETARIA
Isaac Monteiro,30.0
Letícia das Neves,20.0
Dr. João Gabriel Araújo,50.0
Pedro Henrique Ferreira,50.0
Alexia Pinto,20.0
Maria Fernanda Silveira,30.0
Lucca Vieira,20.0
Vitor Hugo Moraes,30.0
Srta. Sophie Pires,10.0
Luigi Moreira,20.0


Essa ação de selecionar um dado trabalhado, recebe vários nomes, como mapeamento
ou projeção.

## Exercício 2

Traga 10 produtos e seu preço convertido para dólar, considere a cotação do dólar
com sendo US\\$1,00=R\\$5,20

In [18]:
%%sql

SELECT produto, (preço/5.20)  AS preço_convertido
FROM produtos LIMIT 10

 * sqlite:///TesteSQL.db
Done.


produto,preço_convertido
prod 00,1.6057692307692306
prod 01,1.8326923076923074
prod 02,0.6288461538461538
prod 03,1.4288461538461537
prod 04,0.0423076923076923
prod 05,1.25
prod 06,1.8865384615384615
prod 07,0.37307692307692303
prod 08,1.3038461538461539
prod 09,1.2538461538461536


# Where

O `LIMIT` como dito anteriormente é a forma mais simples de reduzir a quantidade
de dados trazidos em uma pesquisa, mas a forma mais comum de se usar é o `WHERE`.

Geralmente quando fazemos uma consulta no banco de dados, não precisamos de todos
os dados da tabela, somente dados que respeitem algum requisito, como por exemplo,
no nosso caso, se a gente quiser saber quais clientes temos com menos de 30 anos:

In [19]:
%%sql

SELECT nome, idade FROM clientes WHERE idade < 30

 * sqlite:///TesteSQL.db
Done.


nome,idade
Letícia das Neves,21
Alexia Pinto,21
Lucca Vieira,25
Srta. Sophie Pires,19
Luigi Moreira,22
Sr. João Melo,21
Luigi Cunha,20
Joana Porto,18


## Exercício 3

Traga todos os produtos que custem menos do que 1 real

In [78]:
%%sql

SELECT produto, preço FROM produtos WHERE preço<1

 * sqlite:///TesteSQL.db
Done.


produto,preço
prod 04,0.22
prod 17,0.51
prod 33,0.09
prod 34,0.91
prod 45,0.8
prod 51,0.06
prod 52,0.03
prod 61,0.39
prod 65,0.33
prod 69,0.66


# Join

Uma das principais características dos bancos relacionais é justamente a sua capacidade
de ter relacionamento entre os dados.

Geralmente esse relacionamento é feito utilizando chaves primárias (PK). Geralmente
as PKs são representadas por um número inteiro chamado ID, mas não necessariamente,
por exemplo, um cadastro de pessoas pode ter como PK o CPF da pessoa, o que importa
é que não haja repetição de valores.

As PKs também podem ter mais de uma coluna, por exemplo, um supermercado pode ter
como PK um produto e a marca dele.

Mas por hora iremos abstrair essas PKs menos ortodoxas e focar nas padrões que é só
uma coluna ID.

As PKs geralmente são utilizadas para formar relacionamento entre as tabelas, ou seja,
a partir do momento que eu cadastrei um cliente no meu sistema, e gerei um ID para ele
toda as interações dele serão registradas utilizando o seu ID e não mais dados
pessoais.

No nosso caso, registramos as compras utilizando o ID do cliente e do produto, como
podemos ver abaixo

In [21]:
%%sql

SELECT cl.*, po.*
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
LIMIT 5

 * sqlite:///TesteSQL.db
Done.


id,nome,email,idade,id_1,produto,preço
17,Pedro Pinto,mirella55@ig.com.br,30,7,prod 07,1.94
11,Antônio Fernandes,fvieira@ig.com.br,32,74,prod 74,4.56
0,Isaac Monteiro,caldeiraandre@yahoo.com.br,38,65,prod 65,0.33
12,Sr. João Melo,luna47@ig.com.br,21,20,prod 20,1.94
13,Luigi Cunha,alice99@uol.com.br,20,50,prod 50,4.71


A tabela de compras traz consigo o ID do cliente que fez a compra e o ID do produto
comprado, ocupando assim muito menos espaço do que se eu tivesse uma tabela com
todos os dados.

> NOTA: Quando nas colunas selecionadas faço algo do tipo alias.* eu estou trazendo todas as colunas daquela tabel

> NOTA: Em 99% dos casos, independetemente do intuito do SELECT, o relacionamento entre as tabelas vai utilizar as mesmas colunas que usou em SELECTs anteriores

## Exercício 4

Traga 10 compras do produto `prod 02`

In [22]:
%%sql

SELECT cl.*, po.*
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
WHERE produto = 'prod 02'
LIMIT 10

 * sqlite:///TesteSQL.db
Done.


id,nome,email,idade,id_1,produto,preço
1,Letícia das Neves,cardosohelena@gmail.com,21,2,prod 02,3.27
2,Dr. João Gabriel Araújo,carvalholuana@hotmail.com,52,2,prod 02,3.27


# Distinct

Muitas vezes as nossas consultas nos trazem resultados repetidos, quando a repetição
não nos importa, por exemplo, mesmo na nossa base tendo 20 clientes, não necessariamente
todos efetuaram compras, se eu quiser saber todos que o fizeram posso fazer:

In [23]:
%%sql

SELECT DISTINCT cl.nome
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID

 * sqlite:///TesteSQL.db
Done.


nome
Pedro Pinto
Antônio Fernandes
Isaac Monteiro
Sr. João Melo
Luigi Cunha
Marcos Vinicius Araújo
Maria Fernanda Silveira
Otávio das Neves
Luiza Alves
Pedro Henrique Ferreira


Como podemos ver, apesar de eu ter feito {{N_COMPRAS}} compras, o meu resultado
não traz essa quantidade de linhas, traz exatamente a quantidade de clientes 
que fizeram compras.

## Exercício 5

Liste todos clientes que compraram o produto `prod 02`

In [24]:
%%sql

SELECT DISTINCT cl.nome
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
WHERE produto = 'prod 02'


 * sqlite:///TesteSQL.db
Done.


nome
Letícia das Neves
Dr. João Gabriel Araújo


# Redução

No SQL também são comuns funções de redução, elas percorrem todas as linhas de uma
determinada coluna aplicando uma função de redução (somatório, contagem e etc.)
e retornam somente uma linha ao invés de todas as linhas lidas por ela, por exemplo
se eu quiser saber quantas compras do `prod 02` foram feitas, posso fazer:

In [25]:
%%sql

SELECT count(*)
FROM compras AS co
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
WHERE po.produto = 'prod 02'

 * sqlite:///TesteSQL.db
Done.


count(*)
2


Existem outras funções como SUM, AVG e etc., você pode sempre consultar a 
[documentação oficial](https://www.sqlite.org/lang_aggfunc.html) para ver 
todas elas

## Exercício 6

Qual foi a receita total?

In [26]:
%%sql

SELECT round(sum(po.preço)) AS receita_total
FROM compras AS co
JOIN produtos as po
ON co.ID_PRODUTO = po.ID

 * sqlite:///TesteSQL.db
Done.


receita_total
2281.0


# Group by

As funções de agregação ficam ainda mais fortes quando agrupamos valores.
Isto é, elas mostram todo o seu potencial quando criamos pequenos grupos de dados,
ao invés de contarmos por exemplo todas as vendas do dia, podemos aplicar contagem
produto a produto com somente uma query, como mostrado abaixo:

In [27]:
%%sql

SELECT po.produto, count(*)
FROM compras AS co
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by po.produto
limit 10

 * sqlite:///TesteSQL.db
Done.


produto,count(*)
prod 00,4
prod 02,2
prod 03,4
prod 04,4
prod 05,3
prod 06,6
prod 07,6
prod 08,6
prod 09,7
prod 10,5


Dessa forma, consigo fazer uma contagem diferente para cada produto, sem precisar
fazer diversas querys, algo muito útil.

## Exercício 7

Quanto cada cliente gastou?

In [28]:
%%sql

SELECT cl.nome, round(sum(po.preço)) as gasto
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by cl.nome


 * sqlite:///TesteSQL.db
Done.


nome,gasto
Alexia Pinto,117.0
Antônio Fernandes,99.0
Dr. João Gabriel Araújo,144.0
Isaac Monteiro,96.0
Joana Porto,122.0
Letícia das Neves,100.0
Lucca Vieira,84.0
Luigi Cunha,75.0
Luigi Moreira,145.0
Luiza Alves,127.0


# Order by

Muitas vezes queremos os dados em ordem específicas, por exemplo, ao visitar uma
loja online, você não espera ver produtos caros e desinteressantes sendo exibidos
para você aleatoriamente, você espera ver em uma ordem lógica, sejam os mais
baratos primeiros, os mais relevantes para o seu perfil, ou até alfabéticamente.


Tudo isso pode ser feito utilizando o comando `ORDER BY`, mas misturado com
outras operações como o `LIMIT` ele pode fazer coisas ainda melhores, como
por exemplo, trazer os 10 produtos mais caros:

In [29]:
%%sql

SELECT produto, preço
FROM produtos
ORDER BY preço DESC
LIMIT 10

 * sqlite:///TesteSQL.db
Done.


produto,preço
prod 06,9.81
prod 72,9.7
prod 16,9.69
prod 64,9.68
prod 10,9.63
prod 01,9.53
prod 95,9.39
prod 58,8.94
prod 19,8.87
prod 55,8.66


Com isso podemos ver o poder do `ORDER BY` + `LIMIT` para trazer os N primeiros
registros de uma determinada métrica

## Exercício 8

Traga os 10 clientes que mais gastaram

In [30]:
%%sql

SELECT cl.nome
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by cl.nome
order by sum(po.preço)
LIMIT 10

 * sqlite:///TesteSQL.db
Done.


nome
Luigi Cunha
Nina Lima
Sr. Breno da Cruz
Lucca Vieira
Isaac Monteiro
Antônio Fernandes
Letícia das Neves
Pedro Henrique Ferreira
Vitor Hugo Moraes
Marcos Vinicius Araújo


O order by junto do limit é extremamente importante para fazer paginação de dados,
por exemplo, novamente ao acessar o site de compras, os produtos não são todos
exibidos de uma vez para você, do contrário, a memória do seu computador iria
para o espaço, para evitar isso, tem aqueles botões na parte de baixo para ver
a próxima página, e com isso, você pode avançar e recuar.

Digamos que um cliente viu os 10 produtos mais caros e não se interessou por
nenhum e quer ver a próxima página, ele pode fazer:

In [31]:
%%sql

SELECT produto, preço
FROM produtos
ORDER BY preço DESC
LIMIT 10,10

 * sqlite:///TesteSQL.db
Done.


produto,preço
prod 77,8.48
prod 67,8.47
prod 96,8.39
prod 00,8.35
prod 57,8.22
prod 14,8.19
prod 94,8.19
prod 37,7.97
prod 13,7.87
prod 36,7.86


O primeiro 10, representa de onde começa e o segundo quantos registros vai trazer.
Você pode mexer nos valores acima e testar a vontade para ver como muda.

## Exercício 9

Traga o 11º cliente que mais gastou

In [32]:
%%sql

SELECT cl.nome
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by cl.nome
order by sum(po.preço)
LIMIT 10,01

 * sqlite:///TesteSQL.db
Done.


nome
Otávio das Neves


# Funções janeladas

As funções janeladas ou funções analíticas como também são conhecidas, são funções
que se aplicam a todas a coluna trazendo métricas como qual posição essa pessoa está
entre as que mais gastaram, ou quantas pessoas no mínimo foram necessárias para 
alcançar até 25% da receita total.

In [33]:
%%sql

SELECT produto, preço, rank() over (order by preço desc) as posição
FROM produtos
order by posição
limit 10

 * sqlite:///TesteSQL.db
Done.


produto,preço,posição
prod 06,9.81,1
prod 72,9.7,2
prod 16,9.69,3
prod 64,9.68,4
prod 10,9.63,5
prod 01,9.53,6
prod 95,9.39,7
prod 58,8.94,8
prod 19,8.87,9
prod 55,8.66,10


Você pode usar as funções janeladas quando lhe for conveniente, mas não serão
obrigatórias em nenhum exercício

## Exercício 10

Qual foi o produto que menos vendeu?

In [34]:
%%sql
SELECT produto, rank() over (order by sum(po.preço) desc ) as menos_vendido
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID

 * sqlite:///TesteSQL.db
Done.


produto,menos_vendido
prod 07,1


# Operadores lógicos comuns

Alguns operadores lógicos bem famosos são o `=`, `>`, `<`, `>=`, `<=`, `!=`,
mas existem outros menos comuns, como o `LIKE` e `IN`.

O primeiro, serve para fazer pesquisas simples dentro de texto, ele irá verificar
caracter a caracter se a string informada bate com o padrão, de forma mais prática

```SQL
'abc' LIKE 'abc' -- Verdadeiro

'abcd' LIKE 'abc' -- Falso

'cbc' LIKE 'abc' -- Falso

'dabc' LIKE 'abc' -- Falso
```

Até aí é exatamente igual ao operador `=`, mas quando adicionamos os wildcards
`%` e `_` que o `LIKE` mostra o seu valor.

O wildcard `_` pode ser usado para representar qualquer caracter, por exemplo:

```SQL
'abc' LIKE 'ab_' -- Verdadeiro
'abd' LIKE 'ab_' -- Verdadeiro
'ab0' LIKE 'ab_' -- Verdadeiro
'ab ' LIKE 'ab_' -- Verdadeiro
'abcd' LIKE 'ab_' -- Falso
'cbc' LIKE 'ab_' -- Falso
'ab' LIKE 'ab_' -- Falso
'dabc' LIKE 'ab_' -- Falso
```

Já o wildcard `%` pode ser usado para representar qualquer sequência de caracteres,
por exemplo:

```SQL
'abc' LIKE 'ab%' -- Verdadeiro
'abd' LIKE 'ab%' -- Verdadeiro
'ab0' LIKE 'ab%' -- Verdadeiro
'ab ' LIKE 'ab%' -- Verdadeiro
'abcd' LIKE 'ab%' -- Verdadeiro
'cbc' LIKE 'ab%' -- Falso
'ab' LIKE 'ab%' -- Verdadeiro
'dabc' LIKE 'ab%' -- Falso
```

Usando o nosso exemplo, vamos pegar todos os clientes que tenham e-mail uol:

In [35]:
%%sql

SELECT *
FROM CLIENTES
WHERE EMAIL LIKE '%@uol.com.br'

 * sqlite:///TesteSQL.db
Done.


id,nome,email,idade
3,Pedro Henrique Ferreira,pedro93@uol.com.br,58
6,Lucca Vieira,giovanna58@uol.com.br,25
13,Luigi Cunha,alice99@uol.com.br,20
14,Joana Porto,anthonypeixoto@uol.com.br,18
16,Nina Lima,danilocarvalho@uol.com.br,52


Já o operador `IN` serve para verificar se o valor está dentro de um cojunto, por exemplo

```SQL
1 in (1, 2, 3) -- Verdadeiro
2 in (1, 2, 3) -- Verdadeiro
3 in (1, 2, 3) -- Verdadeiro
4 in (1, 2, 3) -- Falso
(1, 2) in ((1, 2), (3, 4)) -- Verdadeiro
(1, 3) in ((1, 2), (3, 4)) -- Falso
```

In [36]:
%%sql

SELECT *
FROM PRODUTOS
WHERE PRODUTO IN ('prod 01', 'prod 20', 'batata')

 * sqlite:///TesteSQL.db
Done.


id,produto,preço
1,prod 01,9.53
20,prod 20,1.94


# Subquerys

Subquerys são utilizadas para fazer extrações mais complexas, quando para 
alcançar um resultado, é necessário outro resultado, digamos, listar todos os
produtos comprados pelo cliente que mais gastou, primeiro precisamos encotrar o 
cliente que mais gastou e depois os produtos que ele comprou.

Podemos separar isso em duas querys:

1. Selecionar o cliente que mais gastou
2. Selecionar todos os produtos que um cliente comprou

E elas ficariam como:

In [37]:
%%sql

-- Selecionando o cliente que mais gastou

SELECT cl2.ID, cl2.nome, sum(po2.preço) AS GASTO
FROM clientes cl2
JOIN compras AS co2
ON co2.ID_CLIENTE = cl2.ID
JOIN produtos as po2
ON co2.ID_PRODUTO = po2.ID
group by cl2.nome, cl2.id
ORDER BY gasto DESC
LIMIT 1

 * sqlite:///TesteSQL.db
Done.


id,nome,GASTO
5,Maria Fernanda Silveira,181.44


In [38]:
%%sql

-- Selecionar todos os produtos que umm cliente comprou

SELECT DISTINCT po.produto
FROM produtos as po
JOIN compras AS co
ON co.ID_PRODUTO = po.ID
WHERE co.ID_CLIENTE IN (5)

 * sqlite:///TesteSQL.db
Done.


produto
prod 10
prod 68
prod 72
prod 40
prod 47
prod 76
prod 59
prod 38
prod 29
prod 36


In [39]:
%%sql

-- Juntando as duas:
    
SELECT DISTINCT po.produto
FROM produtos as po
JOIN compras AS co
ON co.ID_PRODUTO = po.ID
WHERE co.ID_CLIENTE IN (
    SELECT cl2.ID
    FROM clientes cl2
    JOIN compras AS co2
    ON co2.ID_CLIENTE = cl2.ID
    JOIN produtos as po2
    ON co2.ID_PRODUTO = po2.ID
    group by cl2.nome, cl2.id
    ORDER BY sum(po2.preço) DESC
    LIMIT 1
)

 * sqlite:///TesteSQL.db
Done.


produto
prod 10
prod 68
prod 72
prod 40
prod 47
prod 76
prod 59
prod 38
prod 29
prod 36


Se eu quisesse exibir o nome do cliente, eu poderia fazer mais um join pegando
o nome dele na tabela cliente, mas nesse caso, eu estou pegando um dado que eu
consigo facilmente na subquery, existem formas de trazer dados lá de dentro,
como por  exemplo:

In [40]:
%%sql

    
SELECT tb.nome, po.produto
FROM produtos as po
JOIN compras AS co
ON co.ID_PRODUTO = po.ID
join (
    SELECT cl2.ID, cl2.nome
    FROM clientes cl2
    JOIN compras AS co2
    ON co2.ID_CLIENTE = cl2.ID
    JOIN produtos as po2
    ON co2.ID_PRODUTO = po2.ID
    group by cl2.nome, cl2.id
    ORDER BY sum(po2.preço) DESC
    LIMIT 1
) tb on co.ID_CLIENTE = tb.ID
group by tb.nome, po.produto

 * sqlite:///TesteSQL.db
Done.


nome,produto
Maria Fernanda Silveira,prod 03
Maria Fernanda Silveira,prod 08
Maria Fernanda Silveira,prod 10
Maria Fernanda Silveira,prod 27
Maria Fernanda Silveira,prod 29
Maria Fernanda Silveira,prod 32
Maria Fernanda Silveira,prod 36
Maria Fernanda Silveira,prod 38
Maria Fernanda Silveira,prod 40
Maria Fernanda Silveira,prod 46


Não existe limite de quantidade de subquerys que você pode fazer, sejam uma duas
ou várias, fazendo uma verdadeira boneca russa, mas use com atenção, verifique
sempre se realmente é necessário.

Se por exemplo, eu quisesse saber quantos clientes compraram os mesmos
produtos que a pessoa que mais gastou comprou


1. Selecionar o cliente que mais gastou
2. Selecionar todos os produtos que um cliente comprou
3. Contar quantos clientes compraram um produto

Como a query 1 e 2 já são antigas, vou fazer a 3

In [41]:
%%sql

-- Contar quantos clientes compraram um produto

SELECT po.produto, count(DISTINCT co.ID_CLIENTE)
FROM compras co
JOIN produtos po on po.ID = co.ID_PRODUTO
WHERE co.ID_PRODUTO IN (5)
GROUP by po.produto

 * sqlite:///TesteSQL.db
Done.


produto,count(DISTINCT co.ID_CLIENTE)
prod 05,3


In [42]:
%%sql

SELECT po.produto, count(DISTINCT co.ID_CLIENTE)
FROM compras co
JOIN produtos po on po.ID = co.ID_PRODUTO
WHERE co.ID_PRODUTO IN (
    SELECT DISTINCT po2.id
    FROM produtos as po2
    JOIN compras AS co2
    ON co2.ID_PRODUTO = po2.ID
    WHERE co2.ID_CLIENTE IN (
        SELECT cl3.ID
        FROM clientes cl3
        JOIN compras AS co3
        ON co3.ID_CLIENTE = cl3.ID
        JOIN produtos as po3
        ON co3.ID_PRODUTO = po3.ID
        group by cl3.nome, cl3.id
        ORDER BY sum(po3.preço) DESC
        LIMIT 1
    )
)
GROUP by po.produto

 * sqlite:///TesteSQL.db
Done.


produto,count(DISTINCT co.ID_CLIENTE)
prod 03,3
prod 08,6
prod 10,5
prod 27,4
prod 29,5
prod 32,7
prod 36,8
prod 38,5
prod 40,7
prod 46,8


Mas novamente podemos simplificar usando join ao invés de where, lembre-se,
quanto menos vezes você repetir o nome de uma tabela, melhor

In [43]:
%%sql

SELECT po.produto, count(DISTINCT co.ID_CLIENTE)
FROM compras co
JOIN produtos as po on co.ID_PRODUTO = po.ID
JOIN (
    SELECT cl3.ID
    FROM clientes cl3
    JOIN compras AS co3
    ON co3.ID_CLIENTE = cl3.ID
    JOIN produtos as po3
    ON co3.ID_PRODUTO = po3.ID
    group by cl3.nome, cl3.id
    ORDER BY sum(po3.preço) DESC
    LIMIT 1
) q1
JOIN(
    SELECT co2.id_PRODUTO, co2.ID_CLIENTE
    FROM compras AS co2
    GROUP BY co2.id_PRODUTO, co2.ID_CLIENTE
) q2 ON co.ID_PRODUTO = q2.ID_PRODUTO AND q2.ID_CLIENTE = q1.ID
GROUP by po.produto

 * sqlite:///TesteSQL.db
Done.


produto,count(DISTINCT co.ID_CLIENTE)
prod 03,3
prod 08,6
prod 10,5
prod 27,4
prod 29,5
prod 32,7
prod 36,8
prod 38,5
prod 40,7
prod 46,8


# With

Muitas vezes fica confuso ficar colocando várias subquerys no meio da query 
pricipal, ou você quer repetir ela, não quer ter que trocar alias e etc.

Para resolver esses problemas, ou até outros, consulte a 
[documentação](https://www.sqlite.org/lang_with.html) para ver
tudo o que ele é capaz, existe o `WITH`.

In [44]:
%%sql

WITH q1 AS (
    SELECT cl.ID
    FROM clientes cl
    JOIN compras AS co
    ON co.ID_CLIENTE = cl.ID
    JOIN produtos as po
    ON co.ID_PRODUTO = po.ID
    group by cl.nome, cl.id
    ORDER BY sum(po.preço) DESC
    LIMIT 1
),
q2 AS (
    SELECT co.id_PRODUTO, co.ID_CLIENTE
    FROM compras AS co
    GROUP BY co.id_PRODUTO, co.ID_CLIENTE
)
SELECT po.produto, count(DISTINCT co.ID_CLIENTE)
FROM compras co
JOIN produtos as po on co.ID_PRODUTO = po.ID
JOIN q1
JOIN q2 ON co.ID_PRODUTO = q2.ID_PRODUTO AND q2.ID_CLIENTE = q1.ID
GROUP by po.produto

 * sqlite:///TesteSQL.db
Done.


produto,count(DISTINCT co.ID_CLIENTE)
prod 03,3
prod 08,6
prod 10,5
prod 27,4
prod 29,5
prod 32,7
prod 36,8
prod 38,5
prod 40,7
prod 46,8


## Exercício 11

Traga todos os clientes que compraram o produto que menos vendeu

In [45]:
%%sql


    
SELECT DISTINCT cl.nome, po.produto      #todos os clientes que compraram
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID 
WHERE co.ID_PRODUTO IN (             #colocar o mesmo no select
SELECT  co2.ID_PRODUTO              #produto que menos vendeu
    FROM clientes cl2
    JOIN compras AS co2
    ON co2.ID_CLIENTE = cl2.ID
    JOIN produtos as po2
    ON co2.ID_PRODUTO = po2.ID
    group by po2.produto
    order by  count(*)
    limit 1
)

 * sqlite:///TesteSQL.db
(sqlite3.OperationalError) near "#todos": syntax error
[SQL: SELECT DISTINCT cl.nome, po.produto      #todos os clientes que compraram
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID 
WHERE co.ID_PRODUTO IN (             #colocar o mesmo no select
SELECT  co2.ID_PRODUTO              #produto que menos vendeu
    FROM clientes cl2
    JOIN compras AS co2
    ON co2.ID_CLIENTE = cl2.ID
    JOIN produtos as po2
    ON co2.ID_PRODUTO = po2.ID
    group by po2.produto
    order by  count(*)
    limit 1
)]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


## Exercício 12

Traga quanto gastou cada cliente que comprou o produto que menos vendeu

In [46]:
%%sql

WITH quantidade_de_produto_mv as (         
	SELECT count(*) as qtd
	FROM compras AS co
	group by co.id_produto
	order by qtd
	limit 1	
),
produto_que_menos_vendeu AS (                                 #produto que menos vendeu
	SELECT co.ID_PRODUTO, count(*) as qtd
	FROM compras AS co
	group by co.id_produto
	having qtd = (SELECT qtd from quantidade_de_produto_mv)
),
oq_cd_1_comprou as (                               #o que cada cliente comprou
	SELECT
		cl.nome, 
		co.id_cliente,
		po.produto,
		co.id_produto,
		po.preço 
	FROM clientes cl
	JOIN compras AS co ON co.ID_CLIENTE = cl.ID
	JOIN produtos as po ON co.ID_PRODUTO = po.ID
), quanto_cada_um_gastou as (                          #quanto gastou cada cliente
   SELECT 
    id_cliente,
    nome,
 	sum(preço) as gasto
   FROM oq_cd_1_comprou
   group by id_cliente, nome
 )
 , quem_comprou_menos_vendeu as (               #quem comprou o que menos vendeu
	select 
 	DISTINCT nome, id_cliente 
 	FROM oq_cd_1_comprou as occ
 	JOIN produto_que_menos_vendeu as pmv ON occ.id_produto = pmv.id_produto
 )
  SELECT                                  #quanto gastou cada cliente que comprou o produto que menos vendeu
  qcmv.nome, qcmv.id_cliente, gasto
  FROM quanto_cada_um_gastou as qcug
  JOIN quem_comprou_menos_vendeu as qcmv ON   qcug.id_cliente = qcmv.id_cliente

 * sqlite:///TesteSQL.db
(sqlite3.OperationalError) near "#produto": syntax error
[SQL: WITH quantidade_de_produto_mv as (         
	SELECT count(*) as qtd
	FROM compras AS co
	group by co.id_produto
	order by qtd
	limit 1	
),
produto_que_menos_vendeu AS (                                 #produto que menos vendeu
	SELECT co.ID_PRODUTO, count(*) as qtd
	FROM compras AS co
	group by co.id_produto
	having qtd = (SELECT qtd from quantidade_de_produto_mv)
),
oq_cd_1_comprou as (                               #o que cada cliente comprou
	SELECT
		cl.nome, 
		co.id_cliente,
		po.produto,
		co.id_produto,
		po.preço 
	FROM clientes cl
	JOIN compras AS co ON co.ID_CLIENTE = cl.ID
	JOIN produtos as po ON co.ID_PRODUTO = po.ID
), quanto_cada_um_gastou as (                          #quanto gastou cada cliente
   SELECT 
    id_cliente,
    nome,
 	sum(preço) as gasto
   FROM oq_cd_1_comprou
   group by id_cliente, nome
 )
 , quem_comprou_menos_vendeu as (               #quem comprou o que menos ve

## Exercício 13

Quanto em média os clientes gastaram?

In [47]:
%%sql

SELECT sum(po.preço)/count(distinct co.id_cliente) as média
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID

 * sqlite:///TesteSQL.db
Done.


média
114.05950000000018


## Exercício 14

Quanto em média gastou cada cliente?

In [48]:
%%sql

SELECT cl.nome, sum(po.preço)/count(*) as média
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by cl.nome


 * sqlite:///TesteSQL.db
Done.


nome,média
Alexia Pinto,4.489615384615385
Antônio Fernandes,5.2105263157894735
Dr. João Gabriel Araújo,5.125357142857143
Isaac Monteiro,3.5699999999999994
Joana Porto,4.888400000000002
Letícia das Neves,3.5749999999999997
Lucca Vieira,3.5070833333333336
Luigi Cunha,4.680000000000001
Luigi Moreira,4.5396875
Luiza Alves,5.287916666666667


## Exercício 15

Quantos clientes tem cada faixa etária (Agrupe por faixa de 10 anos)

i.e.: \[18 -> 19\], \[20 -> 29\], \[30 -> 39\], ...

In [76]:
%%sql

SELECT cast(idade/10 AS integer)*10 as faixa_etaria, count(*)
FROM clientes cl
group by faixa_etaria

 * sqlite:///TesteSQL.db
Done.


faixa_etaria,count(*)
10,2
20,6
30,5
40,2
50,5


## Exercício 16

Qual é o gasto médio para cada faixa etária?

In [50]:
%%sql

SELECT cast(idade/10 AS integer)*10 as faixa_etaria, sum(po.preço)/count(distinct co.id_cliente) as gasto_médio
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by faixa_etaria

 * sqlite:///TesteSQL.db
Done.


faixa_etaria,gasto_médio
10,134.66499999999996
20,109.25166666666665
30,121.40600000000002
40,104.36000000000003
50,108.12


## Exercício 17

Quais foram os 5 produtos que mais venderam na faixa etária que teve maior
gasto médio?

In [57]:
%%sql

SELECT po.produto, count(*)            #5 produtos que mais venderam
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by po.produto
order by  count(*) DESC
limit 5


SELECT cast(idade/10 AS integer)*10 as faixa_etaria, sum(po.preço)/count(distinct co.id_cliente) as gasto_médio   # MAIOR GASTO MÉDIO
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by faixa_etaria
order by gasto_médio DESC
limit 1




 * sqlite:///TesteSQL.db
Done.


produto,count(*)
prod 26,12
prod 62,10
prod 46,10
prod 40,9
prod 36,9


## Exercício 18

Quais foram os 5 produtos que mais venderam na faixa etária que teve maior
gasto absoluto?

In [75]:
%%sql

SELECT po.produto, count(*)            #5 produtos que mais venderam
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by po.produto
order by  count(*) DESC
limit 5




SELECT cast(idade/10 AS integer)*10 as faixa_etaria, sum(po.preço) as gasto_absoluto   #MAIOR GASTO ABSOLUTO
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by faixa_etaria
order by gasto_absoluto DESC
limit 1

 * sqlite:///TesteSQL.db
(sqlite3.OperationalError) near "#5": syntax error
[SQL: SELECT po.produto, count(*) #5 produtos que mais venderam
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by po.produto
order by  count(*) DESC
limit 5




SELECT cast(idade/10 AS integer)*10 as faixa_etaria, sum(po.preço) as gasto_absoluto   #MAIOR GASTO ABSOLUTO
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by faixa_etaria
order by gasto_absoluto DESC
limit 1]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


# Desafio

Para cada compra acima de 5 reais, os clientes vão ganhar um número para o
sorteio para cada 5 reais, salve esse número no banco e associe com o cliente

Exemplo, eu gastei 20 reais, vou ganhar 4 números, por exemplo 20, 21, 22 e 23

O primeiro lugar no sorteio vai ser um carro, o segundo uma geladeira
e o terceiro um microondas

E então sorteie e imprima na tela um e-mail para os três vencedores no formato:


```
Para : [e-mail]
Assunto: Parabéns, o número [número] foi sortedo

Corpo: Parabéns Sr(a). [nome], você acaba de ganhar um(a) [item] no nosso sorteio
```
Para esse desafio você pode usar a linguagem `Python` além do `SQL`, 
mas tente usar o banco sempre que possível

Lembre-se do

```python
df = pd.read_sql('SELECT * FROM clientes', db)
```

In [83]:
%%sql

SELECT produto, preço 
FROM produtos 
WHERE preço>5

 * sqlite:///TesteSQL.db
Done.


produto,preço
prod 00,8.35
prod 01,9.53
prod 03,7.43
prod 05,6.5
prod 06,9.81
prod 08,6.78
prod 09,6.52
prod 10,9.63
prod 11,5.5
prod 13,7.87
